In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import xgboost as xgb

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [4]:
# 1. Загрузка очищенных данных
df = pd.read_csv('processed_output.csv', parse_dates=['pickup_date'])
df['ds'] = df['pickup_date'] + pd.to_timedelta(df['pickup_hour'], unit='h')
df = df.set_index('ds')
df

,pickup_location_id,pickup_date,pickup_hour,trip_time,sales_tax,congestion_surcharge,driver_pay,cbd_congestion_fee,passenger_count,trip_distance,...,dropoff_location_id,ride_count,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WT01,WT03
ds,,,,,,,,,,,,,,,,,,,,,
2022-01-07 08:00:00,1.0,2022-01-07,8,NaN,NaN,0.0,NaN,NaN,1.000000,7.35,...,79.500000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-28 15:00:00,1.0,2022-01-28,15,NaN,NaN,0.0,NaN,NaN,1.666667,0.00,...,62.666667,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 04:00:00,1.0,2025-01-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,137.000000,1,3.7,0.0,0.0,0.0,10.6,3.9,NaN,NaN
2025-01-01 05:00:00,1.0,2025-01-01,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,107.000000,1,3.7,0.0,0.0,0.0,10.6,3.9,NaN,NaN
2025-01-01 07:00:00,1.0,2025-01-01,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,150.000000,3,3.7,0.0,0.0,0.0,10.6,3.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-29 09:00:00,265.0,2025-04-29,9,748.0,0.710,0.0,10.540,0.0,NaN,NaN,...,173.666667,6,2.2,0.0,0.0,0.0,26.1,13.3,NaN,NaN
2025-04-29 15:00:00,265.0,2025-04-29,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,161.714286,7,2.2,0.0,0.0,0.0,26.1,13.3,NaN,NaN
2025-04-30 03:00:00,265.0,2025-04-30,3,NaN,NaN,0.0,NaN,0.0,1.000000,0.00,...,201.000000,4,2.6,0.0,0.0,0.0,22.8,15.0,NaN,NaN


In [5]:
# Метрики вручную
def r2_score_np(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res/ss_tot

def rmse_np(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

In [10]:
# После того, как вы прочитали df и установили индекс ds:

# 1. Правильный таргет: агрегируем ride_count по ds
y = df['ride_count'].groupby(df.index).sum().sort_index()

# 2. Экзогенные числовые тоже агрегируем по ds
exog = df.drop(columns=['pickup_date','pickup_hour','ride_count'])
num_cols = exog.select_dtypes(include=[np.number]).columns
exog_num = exog[num_cols].groupby(exog.index).mean()

# 3. Категориальные one-hot + mean по ds (если есть)
cat_cols = exog.select_dtypes(exclude=[np.number]).columns.tolist()
if cat_cols:
    dummies = pd.get_dummies(exog[cat_cols], drop_first=True)
    exog_cat = dummies.groupby(dummies.index).mean()
    exog_agg = pd.concat([exog_num, exog_cat], axis=1).sort_index()
else:
    exog_agg = exog_num.sort_index()
    
# 5. Очистка inf и NaN
exog_agg = exog_agg.replace([np.inf, -np.inf], np.nan)
exog_agg = exog_agg.fillna(method='ffill').fillna(method='bfill').fillna(0)

# 4. Синхронизируем и смотрим
data = pd.concat([y, exog_agg], axis=1).dropna()
print("Индекс таргета:", len(y),      "->", y.index.min(), "–", y.index.max())
print("Индекс экзогенных:", len(exog_agg), "->", exog_agg.index.min(), "–", exog_agg.index.max())
print("Общих точек после dropna:", len(data))

y_sync    = data['ride_count']
exog_sync = data.drop(columns='ride_count')

Индекс таргета: 3419 -> 2008-12-31 22:00:00 – 2025-05-01 22:00:00
Индекс экзогенных: 3419 -> 2008-12-31 22:00:00 – 2025-05-01 22:00:00
Общих точек после dropna: 3419


/var/folders/4_/s5nq1m4x3x766j84pv1dwbz80000gn/T/ipykernel_15047/3712197533.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  exog_agg = exog_agg.fillna(method='ffill').fillna(method='bfill').fillna(0)


In [13]:
exog_sync.isna().sum()

pickup_location_id      0
trip_time               0
sales_tax               0
congestion_surcharge    0
driver_pay              0
cbd_congestion_fee      0
passenger_count         0
trip_distance           0
RatecodeID              0
payment_type            0
fare_amount             0
extra                   0
total_amount            0
dropoff_location_id     0
AWND                    0
PRCP                    0
SNOW                    0
SNWD                    0
TMAX                    0
TMIN                    0
WT01                    0
WT03                    0
dtype: int64

In [18]:
# 4) Разбиение на train/test (80/20)
n = len(y)
split = int(0.8 * n)
train_y, test_y       = y.iloc[:split], y.iloc[split:]
train_exog, test_exog = exog_agg.iloc[:split], exog_agg.iloc[split:]


results = {}

In [27]:
# 5) Seasonal Naïve (исправленный)
season = 24 * 7

# full_series — ваш полный синхронный ряд y_sync или y
full_naive = y.shift(season)

# берём прогноз ровно на тестовые метки
naive_pred = full_naive.loc[test_y.index]

# убираем первые season точек, где прогноз отсутствует
mask_valid = ~naive_pred.isna()

# считаем метрики только по валидным точкам
results['SeasonalNaive'] = (
    r2_score_np(test_y[mask_valid].values, naive_pred[mask_valid].values),
    rmse_np(test_y[mask_valid].values, naive_pred[mask_valid].values)
)

In [20]:
# 6) SARIMAX с экзогенными переменными
sar = SARIMAX(
    train_y, 
    exog=train_exog,
    order=(1,0,1),
    seasonal_order=(1,1,1,24),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)
sar_pred = sar.get_forecast(steps=len(test_y), exog=test_exog).predicted_mean
results['SARIMAX'] = (
    r2_score_np(test_y.values, sar_pred.values),
    rmse_np(test_y.values, sar_pred.values)
)

/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index begin

In [22]:
# 7) XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
xgb_model.fit(train_exog, train_y.values)
xgb_pred = xgb_model.predict(test_exog)
results['XGBoost'] = (
    r2_score_np(test_y.values, xgb_pred),
    rmse_np(test_y.values, xgb_pred)
)  # ← вот эта скобка закрывает кортеж

In [23]:
# 8) RandomForest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(train_exog, train_y.values)
rf_pred = rf_model.predict(test_exog)
results['RandomForest'] = (
    r2_score_np(test_y.values, rf_pred),
    rmse_np(test_y.values, rf_pred)
)


In [25]:
# 9) MLPRegressor
mlp_model = MLPRegressor(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
mlp_model.fit(train_exog, train_y.values)
mlp_pred = mlp_model.predict(test_exog)
results['MLPRegressor'] = (
    r2_score_np(test_y.values, mlp_pred),
    rmse_np(test_y.values, mlp_pred)
)

/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [28]:
# 10) Вывод результатов
print("\nСравнение моделей:")
print("Модель            \tR2\t\tRMSE")
for name,(r2,rmse) in results.items():
    print(f"{name:15s}\t{r2:.3f}\t\t{rmse:.3f}")


Сравнение моделей:
Модель            	R2		RMSE
SeasonalNaive  	0.704		1598.825
SARIMAX        	0.784		1366.914
XGBoost        	0.603		1852.270
RandomForest   	0.651		1736.285
MLPRegressor   	-2.269		5315.530


In [29]:
import joblib

# Пусть ваша лучшая модель SARIMAX называется `sar`
# Сохраним её на диск:
joblib.dump(sar, 'best_sarimax_model.joblib')

# Проверим, что файл создался и что мы можем загрузить модель:
loaded_sar = joblib.load('best_sarimax_model.joblib')

# И сделаем быстрый прогноз, чтобы убедиться, что всё работает:
pred = loaded_sar.get_forecast(steps=len(test_y), exog=test_exog).predicted_mean
print(pred.iloc[:5])

2735    10285.944651
2736    10147.706272
2737    10686.322741
2738     9411.133131
2739    10581.577896
Name: predicted_mean, dtype: float64


/Users/yulialebedeva/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
